# HistGradientBoostingRegressor
## Target Variable: lqt2_penetranceBayesian

In [ ]:
#!pip install sklearn.inspection

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
from sklearn.linear_model import LinearRegression
from sklearn.experimental import enable_hist_gradient_boosting
from sklearn.ensemble import HistGradientBoostingRegressor
from sklearn.datasets import make_regression
from sklearn.experimental import enable_hist_gradient_boosting
from sklearn.ensemble import HistGradientBoostingRegressor
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score, explained_variance_score, accuracy_score
from sklearn.model_selection import train_test_split
#from sklearn.inspection import plot_partial_dependence
from scipy import stats
from datetime import datetime
from sklearn import preprocessing
from sklearn.model_selection import KFold
from sklearn.linear_model import LinearRegression

/usr/local/lib/python3.9/dist-packages/sklearn/experimental/enable_hist_gradient_boosting.py:16: UserWarning: Since version 1.0, it is not needed to import enable_hist_gradient_boosting anymore. HistGradientBoostingClassifier and HistGradientBoostingRegressor are now stable and can be normally imported from sklearn.ensemble.
  warnings.warn(


In [ ]:
!pip install pyspark 
!pip install -U -q PyDrive
!apt install openjdk-8-jdk-headless -qq
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
openjdk-8-jdk-headless is already the newest version (8u362-ga-0ubuntu1~20.04.1).
0 upgraded, 0 newly installed, 0 to remove and 24 not upgraded.


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import os
cur_path = "/content/drive/MyDrive/Capstone/"
os.chdir(cur_path)
!pwd

/content/drive/MyDrive/Capstone


In [ ]:
# Install pyreadr library
!pip install pyreadr

# Import libraries
import pandas as pd
import pyreadr

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
# Read R file into pandas DataFrame
result = pyreadr.read_r('all_lqt2_data_trunc.Rdata')

In [ ]:
df = result['herg.combined.data']

In [ ]:
list(df)

['var',
 'isoform',
 'mut_type',
 'resnum',
 'lqt2',
 'unaff',
 'total_carriers',
 'gnomAD',
 'gnomAD_seq',
 'weight',
 'penetrance_lqt2',
 'lqt2_penetranceBayesian_initial',
 'lqt2_penetranceBayesian',
 'pph2_prob',
 'blast_pssm',
 'provean_score',
 'pamscore',
 'aasimilaritymat',
 'revel_score',
 'hm_ssPeak',
 'hm_tailPeak',
 'hm_vhalfact',
 'hm_vhalfinact',
 'hm_recovfrominact',
 'hm_taudeact_fast',
 'ht_ssPeak',
 'ht_tailPeak',
 'ht_vhalfact',
 'ht_vhalfinact',
 'ht_recovfrominact',
 'ht_taudeact_fast',
 'lqt2_dist',
 'lqt2_dist_weight',
 'Structure',
 'Function',
 'p_mean_w',
 'prior_mean_w',
 'prior_mean',
 'alpha',
 'beta',
 'lqt2_patho',
 'p_mean_prior',
 'RMSF',
 'lqt2_dist_sequence',
 'lqt2_dist_weight_sequence',
 'lqt2_dist_observed',
 'lqt2_dist_weight_observed',
 'lqt2_dist_max',
 'lqt2_dist_weight_max',
 'lqt2_dist_mean',
 'lqt2_dist_weight_mean',
 'lqt2_dist_obs_max',
 'lqt2_dist_weight_obs_max',
 'lqt2_dist_obs_mean',
 'lqt2_dist_weight_obs_mean',
 'lqt2_dist_obs_seq',


#### MSE: mean squared error
#### RMSE: root-mean-square deviation
#### MAE: mean absolute error

In [ ]:
filter_columns = ['lqt2_penetranceBayesian','RMSF','lqt2_dist_sequence','lqt2_dist_weight_sequence',
                  'lqt2_dist_observed','lqt2_dist_weight_observed', 'lqt2_dist_max', 
                  'lqt2_dist_weight_max','lqt2_dist_mean','lqt2_dist_weight_mean',
                  'lqt2_dist_obs_max','lqt2_dist_weight_obs_max','lqt2_dist_obs_mean',
                  'lqt2_dist_weight_obs_mean','lqt2_dist_obs_seq','lqt2_dist_weight_obs_seq', 
                  'revel_score','ht_tailPeak','hm_tailPeak','lqt2_dist','lqt2_dist_weight',
                  'lqt2','unaff','total_carriers','gnomAD','gnomAD_seq','weight','pph2_prob',
                  'blast_pssm','provean_score','pamscore','aasimilaritymat','hm_ssPeak',
                  'hm_vhalfact','hm_vhalfinact','hm_recovfrominact','hm_taudeact_fast',
                  'ht_ssPeak','ht_vhalfact','ht_vhalfinact','ht_recovfrominact',
                  'ht_taudeact_fast','prior_mean_w', 'prior_mean','alpha','beta','lqt2_patho',
                  'p_mean_prior']
df = df.filter(filter_columns, axis=1)

In [ ]:
features = ['RMSF','lqt2_dist','lqt2_dist_observed','lqt2_dist_max', 'lqt2_dist_mean',
                  'lqt2_dist_obs_max','lqt2_dist_obs_mean', 'revel_score']
 
X = pd.DataFrame(df[features])
y = pd.DataFrame(df['lqt2_penetranceBayesian'])

In [ ]:
# split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# create and train the HistGradientBoostingRegressor model
reg = HistGradientBoostingRegressor(max_iter=100, max_depth=3, random_state=42)
reg.fit(X_train, y_train)

# evaluate the model on the test set
y_pred = reg.predict(X_test)
mse = mean_squared_error(y_test, y_pred)
rmse = np.sqrt(mse)
mae = mean_absolute_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)
evs = explained_variance_score(y_test, y_pred)

print("MSE: {:.2f}".format(mse))
print("RMSE: {:.2f}".format(rmse))
print("MAE: {:.2f}".format(mae))
print("R-squared: {:.2f}".format(r2))
print("Explained variance score: {:.2f}".format(evs))

/usr/local/lib/python3.9/dist-packages/sklearn/utils/validation.py:1143: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


MSE: 0.02
RMSE: 0.15
MAE: 0.12
R-squared: 0.51
Explained variance score: 0.52


In [ ]:
print("Training dataset shape: ", X_train.shape, y_train.shape)
print("Testing dataset shape: ", X_test.shape, y_test.shape)

Training dataset shape:  (720, 8) (720, 1)
Testing dataset shape:  (180, 8) (180, 1)


### --------------------------------------------------------------------------------------------------------------------------------------------------------------

In [ ]:
features = ['RMSF','lqt2_dist','lqt2_dist_max', 'lqt2_dist_mean', 'revel_score']

X = pd.DataFrame(df[features])
y = pd.DataFrame(df['lqt2_penetranceBayesian'])

In [ ]:
# split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# create and train the HistGradientBoostingRegressor model
reg = HistGradientBoostingRegressor(max_iter=100, max_depth=3, random_state=42)
reg.fit(X_train, y_train)

# evaluate the model on the test set
y_pred = reg.predict(X_test)
mse = mean_squared_error(y_test, y_pred)
rmse = np.sqrt(mse)
mae = mean_absolute_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)
evs = explained_variance_score(y_test, y_pred)

print("MSE: {:.2f}".format(mse))
print("RMSE: {:.2f}".format(rmse))
print("MAE: {:.2f}".format(mae))
print("R-squared: {:.2f}".format(r2))
print("Explained variance score: {:.2f}".format(evs))

/usr/local/lib/python3.9/dist-packages/sklearn/utils/validation.py:1143: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


MSE: 0.02
RMSE: 0.15
MAE: 0.12
R-squared: 0.51
Explained variance score: 0.51


### --------------------------------------------------------------------------------------------------------------------------------------------------------------

In [ ]:
features = ['RMSF','lqt2_dist_observed', 'lqt2_dist_obs_max','lqt2_dist_obs_mean', 'revel_score']

X = pd.DataFrame(df[features])
y = pd.DataFrame(df['lqt2_penetranceBayesian'])

In [ ]:
# split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# create and train the HistGradientBoostingRegressor model
reg = HistGradientBoostingRegressor(max_iter=100, max_depth=3, random_state=42)
reg.fit(X_train, y_train)

# evaluate the model on the test set
y_pred = reg.predict(X_test)
mse = mean_squared_error(y_test, y_pred)
rmse = np.sqrt(mse)
mae = mean_absolute_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)
evs = explained_variance_score(y_test, y_pred)

print("MSE: {:.2f}".format(mse))
print("RMSE: {:.2f}".format(rmse))
print("MAE: {:.2f}".format(mae))
print("R-squared: {:.2f}".format(r2))
print("Explained variance score: {:.2f}".format(evs))

/usr/local/lib/python3.9/dist-packages/sklearn/utils/validation.py:1143: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


MSE: 0.02
RMSE: 0.15
MAE: 0.12
R-squared: 0.52
Explained variance score: 0.52


### --------------------------------------------------------------------------------------------------------------------------------------------------------------

In [ ]:
features = ['RMSF','lqt2_dist','revel_score']

X = pd.DataFrame(df[features])
y = pd.DataFrame(df['lqt2_penetranceBayesian'])

In [ ]:
# split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# create and train the HistGradientBoostingRegressor model
reg = HistGradientBoostingRegressor(max_iter=100, max_depth=3, random_state=42)
reg.fit(X_train, y_train)

# evaluate the model on the test set
y_pred = reg.predict(X_test)
mse = mean_squared_error(y_test, y_pred)
rmse = np.sqrt(mse)
mae = mean_absolute_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)
evs = explained_variance_score(y_test, y_pred)

print("MSE: {:.2f}".format(mse))
print("RMSE: {:.2f}".format(rmse))
print("MAE: {:.2f}".format(mae))
print("R-squared: {:.2f}".format(r2))
print("Explained variance score: {:.2f}".format(evs))

/usr/local/lib/python3.9/dist-packages/sklearn/utils/validation.py:1143: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


MSE: 0.02
RMSE: 0.15
MAE: 0.12
R-squared: 0.51
Explained variance score: 0.51


### --------------------------------------------------------------------------------------------------------------------------------------------------------------

In [ ]:
features = ['RMSF','lqt2_dist_max','revel_score']

X = pd.DataFrame(df[features])
y = pd.DataFrame(df['lqt2_penetranceBayesian'])

In [ ]:
# split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# create and train the HistGradientBoostingRegressor model
reg = HistGradientBoostingRegressor(max_iter=100, max_depth=3, random_state=42)
reg.fit(X_train, y_train)

# evaluate the model on the test set
y_pred = reg.predict(X_test)
mse = mean_squared_error(y_test, y_pred)
rmse = np.sqrt(mse)
mae = mean_absolute_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)
evs = explained_variance_score(y_test, y_pred)

print("MSE: {:.2f}".format(mse))
print("RMSE: {:.2f}".format(rmse))
print("MAE: {:.2f}".format(mae))
print("R-squared: {:.2f}".format(r2))
print("Explained variance score: {:.2f}".format(evs))

MSE: 0.02
RMSE: 0.15
MAE: 0.12
R-squared: 0.53
Explained variance score: 0.53


/usr/local/lib/python3.9/dist-packages/sklearn/utils/validation.py:1143: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


### --------------------------------------------------------------------------------------------------------------------------------------------------------------

In [ ]:
features = ['RMSF','lqt2_dist_mean','revel_score']

X = pd.DataFrame(df[features])
y = pd.DataFrame(df['lqt2_penetranceBayesian'])
y = np.ravel(y)

In [ ]:
# split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# create and train the HistGradientBoostingRegressor model
reg = HistGradientBoostingRegressor(max_iter=100, max_depth=3, random_state=42)
reg.fit(X_train, y_train)

# evaluate the model on the test set
y_pred = reg.predict(X_test)
mse = mean_squared_error(y_test, y_pred)
rmse = np.sqrt(mse)
mae = mean_absolute_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)
evs = explained_variance_score(y_test, y_pred)

print("MSE: {:.2f}".format(mse))
print("RMSE: {:.2f}".format(rmse))
print("MAE: {:.2f}".format(mae))
print("R-squared: {:.2f}".format(r2))
print("Explained variance score: {:.2f}".format(evs))

MSE: 0.02
RMSE: 0.15
MAE: 0.12
R-squared: 0.52
Explained variance score: 0.52


### --------------------------------------------------------------------------------------------------------------------------------------------------------------

In [ ]:
features = ['RMSF', 'lqt2_dist_observed', 'revel_score']

X = pd.DataFrame(df[features])
y = pd.DataFrame(df['lqt2_penetranceBayesian'])
y = np.ravel(y)

In [ ]:
# split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# create and train the HistGradientBoostingRegressor model
reg = HistGradientBoostingRegressor(max_iter=100, max_depth=3, random_state=42)
reg.fit(X_train, y_train)

# evaluate the model on the test set
y_pred = reg.predict(X_test)
mse = mean_squared_error(y_test, y_pred)
rmse = np.sqrt(mse)
mae = mean_absolute_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)
evs = explained_variance_score(y_test, y_pred)

print("MSE: {:.2f}".format(mse))
print("RMSE: {:.2f}".format(rmse))
print("MAE: {:.2f}".format(mae))
print("R-squared: {:.2f}".format(r2))
print("Explained variance score: {:.2f}".format(evs))

MSE: 0.02
RMSE: 0.15
MAE: 0.12
R-squared: 0.52
Explained variance score: 0.52


### --------------------------------------------------------------------------------------------------------------------------------------------------------------

In [ ]:
features = ['RMSF', 'lqt2_dist_obs_max', 'revel_score']

X = pd.DataFrame(df[features])
y = pd.DataFrame(df['lqt2_penetranceBayesian'])
y = np.ravel(y)

In [ ]:
# split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# create and train the HistGradientBoostingRegressor model
reg = HistGradientBoostingRegressor(max_iter=100, max_depth=3, random_state=42)
reg.fit(X_train, y_train)

# evaluate the model on the test set
y_pred = reg.predict(X_test)
mse = mean_squared_error(y_test, y_pred)
rmse = np.sqrt(mse)
mae = mean_absolute_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)
evs = explained_variance_score(y_test, y_pred)

print("MSE: {:.2f}".format(mse))
print("RMSE: {:.2f}".format(rmse))
print("MAE: {:.2f}".format(mae))
print("R-squared: {:.2f}".format(r2))
print("Explained variance score: {:.2f}".format(evs))

MSE: 0.02
RMSE: 0.15
MAE: 0.12
R-squared: 0.53
Explained variance score: 0.53


### --------------------------------------------------------------------------------------------------------------------------------------------------------------

### --------------------------------------------------------------------------------------------------------------------------------------------------------------

In [ ]:
features = ['RMSF', 'lqt2_dist_obs_mean', 'revel_score']



X = pd.DataFrame(df[features])
y = pd.DataFrame(df['lqt2_penetranceBayesian'])
y = np.ravel(y)

In [ ]:
# split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# create and train the HistGradientBoostingRegressor model
reg = HistGradientBoostingRegressor(max_iter=100, max_depth=3, random_state=42)
reg.fit(X_train, y_train)

# evaluate the model on the test set
y_pred = reg.predict(X_test)
mse = mean_squared_error(y_test, y_pred)
rmse = np.sqrt(mse)
mae = mean_absolute_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)
evs = explained_variance_score(y_test, y_pred)

print("MSE: {:.2f}".format(mse))
print("RMSE: {:.2f}".format(rmse))
print("MAE: {:.2f}".format(mae))
print("R-squared: {:.2f}".format(r2))
print("Explained variance score: {:.2f}".format(evs))

MSE: 0.02
RMSE: 0.14
MAE: 0.12
R-squared: 0.54
Explained variance score: 0.54
